In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import random
import time 
import PIL
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image

from tensorflow.keras.callbacks import CSVLogger

%matplotlib inline

In [ ]:
def test_device():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
    gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    print('GPU Info')
    for gpu in gpus:
        print('   ', gpu)
    print('CPU Info')
    for cpu in cpus:
        print('   ', cpu)
    
    print()
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[0], True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            print(e)
        
test_device()
tf.keras.backend.clear_session() 
AUTOTUNE = tf.data.experimental.AUTOTUNE 

In [18]:
data_root_orig = './data/sep_data/train'
val_data_root_orig = './data/sep_data/validation'
img_size = 224
batch_size = 16
epoch = 50
channel_size = 1
ver = 5

In [19]:
data_root = pathlib.Path(data_root_orig)
print(data_root, list(data_root.iterdir()))
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print(label_to_index)

val_data_root = pathlib.Path(val_data_root_orig)
print(val_data_root, list(val_data_root.iterdir()))
val_label_names = sorted(item.name for item in val_data_root.glob('*/') if item.is_dir())
val_label_to_index = dict((name, index) for index, name in enumerate(label_names))
print(val_label_to_index)

data/sep_data/train [PosixPath('data/sep_data/train/2'), PosixPath('data/sep_data/train/0'), PosixPath('data/sep_data/train/5'), PosixPath('data/sep_data/train/3'), PosixPath('data/sep_data/train/4'), PosixPath('data/sep_data/train/1')]
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}
data/sep_data/validation [PosixPath('data/sep_data/validation/2'), PosixPath('data/sep_data/validation/0'), PosixPath('data/sep_data/validation/5'), PosixPath('data/sep_data/validation/3'), PosixPath('data/sep_data/validation/4'), PosixPath('data/sep_data/validation/1')]
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [20]:
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
image_count = len(all_image_paths)
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]

val_all_image_paths = list(val_data_root.glob('*/*'))
val_all_image_paths = [str(path) for path in val_all_image_paths]
val_image_count = len(val_all_image_paths)
random.shuffle(val_all_image_paths)
val_all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in val_all_image_paths]

print("Train image count      : {}".format(image_count))
print("Validation image count : {}".format(val_image_count))

Train image count      : 12019
Validation image count : 513


In [21]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize(image, [img_size, img_size])
    return tf.keras.applications.nasnet.preprocess_input(image)

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), tf.one_hot(label, depth=6)

In [22]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
image_label_ds = ds.map(load_and_preprocess_from_path_label)
val_ds = tf.data.Dataset.from_tensor_slices((val_all_image_paths, val_all_image_labels))
val_image_label_ds = val_ds.map(load_and_preprocess_from_path_label)

steps_per_epoch = int(image_count / batch_size)
val_steps_per_epoch = int(val_image_count / batch_size)

In [23]:
default_timeit_steps = 2*steps_per_epoch+1

def timeit(ds, steps=default_timeit_steps):
    overall_start = time.time()
    it = iter(ds.take(steps+1))
    next(it)

    start = time.time()
    for i,(images,labels) in enumerate(it):
        if i%10 == 0:
            print('.',end='')
            
    end = time.time()

    duration = end-start
    print("{} batches: {} s".format(steps, duration))
    print("{:0.5f} Images/s".format(batch_size*steps/duration))
    print("Total time: {}s".format(end-overall_start))

In [24]:
ds = image_label_ds.cache()
ds = ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(batch_size).prefetch(buffer_size=AUTOTUNE)
timeit(ds)

.......................................................................................................................................................1503 batches: 0.5255298614501953 s
45759.53103 Images/s
Total time: 39.25286817550659s


In [25]:
val_ds = val_image_label_ds.cache()
val_ds = val_ds.batch(batch_size).prefetch(buffer_size=val_image_count)
timeit(val_ds, 2*val_steps_per_epoch+1)

....65 batches: 1.413372278213501 s
735.82878 Images/s
Total time: 1.472438097000122s


In [26]:
tf.keras.backend.clear_session() 
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(img_size, img_size, channel_size), 
                                               weights=None, include_top=False)
mobile_net.trainable = True

dense_net = tf.keras.applications.DenseNet201(input_shape=(img_size, img_size, channel_size),
                                               weights=None, include_top=False)
dense_net.trainable = True

nas_net = tf.keras.applications.NASNetLarge(input_shape=(img_size, img_size, channel_size),
                                               weights=None, include_top=False)

In [27]:
model_mobile = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(6),
])

In [28]:
model_nas = tf.keras.Sequential([
    nas_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(6)
])

In [29]:
model_dense = tf.keras.Sequential([
    dense_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(6),
])

In [30]:
path = './model/checkpoints_{}_{}_{}_{}'.format(img_size, batch_size, 888, 0)
cp = tf.keras.callbacks.ModelCheckpoint(filepath=path,
                                        monitor='val_loss',
                                        verbose=0, save_best_only=True,
                                        save_weights_only=False, mode='min', save_freq='epoch')

In [ ]:
model_ = model_nas
model_.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

history = model_.fit(ds, epochs=30, steps_per_epoch=steps_per_epoch, 
          validation_data=val_ds, validation_steps=val_steps_per_epoch,
          callbacks=[cp])

Epoch 1/30
751/751 [==============================] - ETA: 0s - loss: 0.4638 - accuracy: 0.8351WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./model/checkpoints_224_16_888_0/assets
751/751 [==============================] - 465s 619ms/step - loss: 0.4638 - accuracy: 0.8351 - val_loss: 2.1086 - val_accuracy: 0.2656
Epoch 2/30
751/751 [==============================] - 367s 489ms/step - loss: 0.2369 - accuracy: 0.9193 - val_loss: 3.1826 - val_accuracy: 0.2656
Epoch 3/30
751/751 [==============================] - 368s 491ms/step - loss: 0.1881 - accuracy: 0.9338 - val_loss: 4.6770 - val_accuracy: 0.2656
Epoch 4/30
751/751 [==========================

In [ ]:
log_ = 'log.log'
with open(log_, 'a+') as log:
    model_name = (path.split('/')[-1])
    valloss = min(history.history['val_loss'])
    idx = history.history['val_loss'].index(valloss)
    valacc = history.history['val_accuracy'][idx]
    log.write("{}, {}, {}\n".format(model_name, valloss, valacc))